In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split
import time
import seaborn as sns
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
import eli5
from eli5.sklearn import PermutationImportance

import tensorflow as tf
from tensorflow import keras


from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel

import category_encoders as ce
import graphviz

from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [ ]:
def filter_years(df):
    """
    filters years 2013 and 2018 which we have to handle.
    """
    df['Date'] = pd.to_datetime(df['Date'])
    df = df[df['Date'].dt.year.isin([2013, 2018])]
    return df

weather = pd.read_csv("weatherAUS.csv")  # read csv data into pandas data frame
weather = filter_years(weather)
weather['Date'].dt.year.value_counts()

# Aufgabe 1 Feature Engineering

In [ ]:
# Spalten mit fehlender Zielvariable "RainTomorrow" rausschmeißen.
# Man könnte diese Auch als Testvariablen nehmen, dann wären diese aber nicht zufällig ausgewählt...
weather = weather[weather['RainTomorrow'].notna()]

In [ ]:
# Spalten, in denen mehr als 40% der Variablen fehlen rausscheißen.
# Zeilen, in denen mehr als 50% der Variablen fehlen rausschmeißen.

weather = weather[weather.columns[weather.isnull().mean() < 0.4]]
weather = weather.loc[weather.isnull().mean(axis=1) < 0.5]

## Null Accuracy

In [ ]:
ax = sns.countplot(x="RainTomorrow", data=weather)

ax.set_xlabel("RainTomorrow", fontsize=18)
ax.set_ylabel('Count', fontsize=18)

plt.xticks(fontsize=18)
plt.yticks(fontsize=18)

plt.savefig('distribution_target_variable.png')

## Feate creation
creating new features

In [ ]:
weather['Year'] = weather['Date'].dt.year  # get year
weather['Month'] = weather['Date'].dt.month  # get month
weather['Day'] = weather['Date'].dt.day  # get day

weather['MinMaxDiff'] = weather['MaxTemp'] - weather['MinTemp']
weather['PressureDiff'] = weather['Pressure3pm'] - weather['Pressure9am']
weather['WindSpeedDiff'] = weather['Pressure3pm'] - weather['WindSpeed9am']
weather['HumidityDiff'] = weather['Humidity3pm'] - weather['Humidity9am']

## Feature Binning

diskretisierung von Features. 

In [ ]:
def encode_season(month):
    if month >= 9 and month <= 11:
        return 'Spring'
    if month == 12 or month <= 2:
        return 'Summer'
    if month >= 3 and month <= 5:
        return 'Autumn'
    if month >= 6 and month <= 8:
        return 'Winter'
    
weather['Season'] = weather['Month'].apply(encode_season)


In [ ]:
# Ist quasy "Target Encoding". Nur halt manuell...

def encode_rainly_month(month):
    rainy_month = [5,6, 7,8,11]
    if month in rainy_month:
        return 1
    return 0

weather['RainyMonth'] = weather['Month'].apply(encode_rainly_month)

## Train Test split
**Important!** 

Before starting Feature Engineering one must split the dataset to ovoid test train leakage!
All Decisions in Data Engineering must be made on the Train Set only! From here, we assume that we dont have any 
knowledge about the test data.

In [ ]:
#Zunächst wird noch nicht die Zielvariable "abgespalten"
#Warum? Wenn die Zielvariable noch im gleichen DataFrame ist, kann man leichter Outlier rausschmeißen.
train, test = train_test_split(weather, test_size=0.2, random_state = 0)

## Outlier detection (Optional)
**important**

this must be done after splitting. Again: We do not know anything about the test data!

**-> Verschlechtert das Ergebnis!**

In [ ]:
plt.figure(figsize=(15,10))

plt.subplot(2, 2, 1)
fig = train.boxplot(column='Rainfall')
fig.set_title('')
fig.set_ylabel('Rainfall')

plt.subplot(2, 2, 2)
fig = train.boxplot(column='WindGustSpeed')
fig.set_title('')
fig.set_ylabel('WindGustSpeed')

In [ ]:
higher_lim = train['Rainfall'].quantile(0.995)
train = train[train['Rainfall'] < higher_lim]
higher_lim = train['WindGustSpeed'].quantile(0.995)
train = train[train['WindGustSpeed'] < higher_lim]

## Train Test Split part 2

hier wird jetzt die Zielvariable abgespalten

In [ ]:
X_train = train.drop(['RainTomorrow'], axis=1)
y_train = train['RainTomorrow']

X_test = test.drop(['RainTomorrow'], axis=1)
y_test = test['RainTomorrow']

## Impute missing Data (Univariat)

In [ ]:
# Impute values the naiive approache without considering the locations or other stuff like season

for dataset in [X_train, X_test]:

    colums_containing_nan = dataset.columns[dataset.isnull().any()]
    
    numerical_containing_nan = [col for col in colums_containing_nan if dataset[col].dtypes != 'O']
    categorial_containing_nan = [col for col in colums_containing_nan if dataset[col].dtypes == 'O']

    for col in numerical_containing_nan:
        col_median=X_train[col].median() #always use median from Train data ! Never impute based on Test Data ! we have to assume we dont know it.
        dataset[col] = dataset[col].fillna(col_median) 
        
    for col in categorial_containing_nan:
        col_most_occuring = X_train[col].mode()[0]
        dataset[col] = dataset[col].fillna(col_most_occuring)     
        

# Encoding Categorial Variables

In [ ]:
X_train['RainToday'] = X_train["RainToday"].replace({'No':0, 'Yes':1})
X_test['RainToday'] = X_test["RainToday"].replace({'No':0, 'Yes':1})

y_train = y_train.replace({'No':0, 'Yes':1})
y_test = y_test.replace({'No':0, 'Yes':1})

### Target encoding

Beim target Encoding kodieren wir die Variable als Einfluss auf die Zielvariable. Wenn es also in Perth zu 20% geregnet hat, dann wird Perth mit 0.2 kodiert.

In [ ]:
# Create the encoder
cat_features=['Location','WindGustDir',"WindDir9am", "WindDir3pm"]
for feature in [cat_features]:
    target_enc = ce.TargetEncoder(cols=feature)
    target_enc.fit(X_train[feature], y_train)

    # Transform the features, rename the columns with _target suffix, and join to dataframe
    X_train = X_train.join(target_enc.transform(X_train[feature]).add_suffix('_target'))
    X_test = X_test.join(target_enc.transform(X_test[feature]).add_suffix('_target'))

### One Hot Encoding

In [ ]:
# apply One Hot encoding

for col in ["Season"]:
    encoded_columns = pd.get_dummies(X_train[col], prefix=col, drop_first=True)
    X_train = X_train.join(encoded_columns).drop(col, axis=1)
    
    encoded_columns = pd.get_dummies(X_test[col], prefix=col, drop_first=True)
    X_test = X_test.join(encoded_columns).drop(col, axis=1)

# Remove Features

Some features where transformed to other columns and can be removed.

In [ ]:
columns_to_drop = ['Date', 'Location','WindGustDir',"WindDir9am", "WindDir3pm"]
X_train.drop(labels=columns_to_drop, axis=1, inplace=True)
X_test.drop(labels=columns_to_drop, axis=1, inplace=True)

## Scale numerical features

**Note !**
This step is not required for decision trees. Still it is recommended to do. Why?
1. We need it for Feature Selection 
2. If we change the tree to a regression or a NN, we need it.
3. It doesnt do any harm.

In [ ]:
scaler = MinMaxScaler()
cols = X_train.columns

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = pd.DataFrame(X_train, columns=cols)
X_test = pd.DataFrame(np.array(X_test), columns=cols)

# Feature Selection


## The univariate way

In [ ]:
# Tamara
# show correlation matrix
train = X_train
train["RainTomorrow"] = y_train.values
train.corr()

In [ ]:
# Tamara
# show correlation matrix as heatmap
plt.imshow(train.corr(), cmap='hot', interpolation='nearest')
plt.show()

In [ ]:
NUM_FEATURES_TO_SELECT = 5

selector = SelectKBest(f_classif, k=NUM_FEATURES_TO_SELECT)

X_new = selector.fit_transform(X_train, y_train)

selected_features = pd.DataFrame(selector.inverse_transform(X_new), 
                                 index=X_train.index, 
                                 columns=X_train.columns)

selected_columns = selected_features.columns[selected_features.var() != 0]

X_train[selected_columns].head()

## The multivariate way (Optional)

In [ ]:
# Regularisation strengh is set by regularization parameter C.
# Note: the lower C, the higher the regularization
logistic = LogisticRegression(C=0.01, penalty="l1", solver='liblinear', max_iter=10000, random_state=7).fit(X_train, y_train)
model = SelectFromModel(logistic, prefit=True)

X_new = model.transform(X_train)

selected_features = pd.DataFrame(model.inverse_transform(X_new), 
                                 index=X_train.index,
                                 columns=X_train.columns)

# Dropped columns have values of all 0s, keep other columns 
selected_columns = selected_features.columns[selected_features.var() != 0]
selected_columns

In [ ]:
# getting accuracy for a logistic regression model 
logistic = LogisticRegression().fit(X_train[selected_columns], y_train)

test_predictions = logistic.predict(X_test[selected_columns]).round().astype(int)
print(accuracy_score(y_test, test_predictions))
mean_absolute_error(y_test, test_predictions)

# Aufgabe 2 Entscheidungsbäume

In [ ]:
model = tree.DecisionTreeClassifier(min_impurity_decrease=0.0001, max_depth=8)
model.fit(X_train, y_train)

test_predictions = model.predict(X_test).round().astype(int)
print(accuracy_score(y_test, test_predictions))
mean_absolute_error(y_test, test_predictions)

In [ ]:
0.7917541229385308
0.20824587706146927

# Visualization

In [ ]:
fig, ax = plt.subplots(figsize = (20,10))
tree.plot_tree(model, ax=ax,
              #feature_names=selected_columns,
              filled = True,
              rounded = True,
              precision =2,
              fontsize = 10,
               class_names=["no Rain", "Rain"]
              );

plt.savefig('treeMinImpurityDecrease.png')

In [ ]:
tree_graph = tree.export_graphviz(model, out_file=None, feature_names=selected_columns, class_names=["no Rain", "Rain"])
graphviz.Source(tree_graph)

In [ ]:
# Tamara
# change max_depth
model1 = tree.DecisionTreeClassifier(max_depth=5)

model1.fit(X_train[selected_columns], y_train)

In [ ]:
# Tamara
test_predictions1 = model1.predict(X_test[selected_columns]).round().astype(int)
print(accuracy_score(y_test, test_predictions1))
mean_absolute_error(y_test, test_predictions1)

In [ ]:
# Tamara
fig, ax = plt.subplots(figsize = (20,10))
tree.plot_tree(model1, ax=ax,
               feature_names=selected_columns,
               class_names= ["Rain", "no Rain"],
               filled = True,
               rounded = True,
               precision = 2,
               fontsize = 10);

In [ ]:
# Tamara
# change min_impurity_increase
model2 = tree.DecisionTreeClassifier(min_impurity_decrease=0.003)

model2.fit(X_train[selected_columns], y_train)

# Tamara
test_predictions2 = model2.predict(X_test[selected_columns]).round().astype(int)
print(accuracy_score(y_test, test_predictions2))
mean_absolute_error(y_test, test_predictions2)

In [ ]:
# Tamara
fig, ax = plt.subplots(figsize = (20,10))
tree.plot_tree(model2, ax=ax,
               feature_names=selected_columns,
               class_names= ["Rain", "no Rain"],
               filled = True,
               rounded = True,
               precision = 2,
               fontsize = 10);

In [ ]:
# Tamara
# change max_leaf_nodes
model3 = tree.DecisionTreeClassifier(criterion="entropy", max_depth=5)

model3.fit(X_train[selected_columns], y_train)

# Tamara
test_predictions3 = model3.predict(X_test[selected_columns]).round().astype(int)
print(accuracy_score(y_test, test_predictions3))
mean_absolute_error(y_test, test_predictions3)

In [ ]:
# Tamara
fig, ax = plt.subplots(figsize = (20,10))
tree.plot_tree(model3, ax=ax,
               feature_names=selected_columns,
               class_names= ["Rain", "no Rain"],
               filled = True,
               rounded = True,
               precision = 2,
               fontsize = 10);

## Analysis

In [ ]:
perm = PermutationImportance(model, random_state=1).fit(X_test, y_test)
eli5.show_weights(perm, feature_names = X_test.columns.tolist())

## cost complexity pruning
Führe mit mind 3 Bäumen unterschiedlicher Tiefe aus Aufgabenteil c) ein Minimal Cost Complexity Pruning durch. Wie verändern dich die Bäume bei Variation des Prunings? Welche Auswirkung auf die Modellgüte hat das?

In [ ]:
# Tamara
model = tree.DecisionTreeClassifier()
path = model.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas, impurities = path.ccp_alphas, path.impurities
ccp_alphas.size

In [ ]:
# Tamara
# take only subset of alphas to speed up further calculations
ccp_alphas_part = ccp_alphas[[0, 68, 136, 204, 272, 340, 408, 476, 544, 612, 680, 748, 816, 884, 952, 1020, 1088, 1156, 1224, 1292, (ccp_alphas.size-20)]]

In [ ]:
#Tamara
fig, ax = plt.subplots()
ax.plot(ccp_alphas[:-1], impurities[:-1], marker='o', drawstyle="steps-post")
ax.set_xlabel("effective alpha")
ax.set_ylabel("total impurity of leaves")
ax.set_title("Total Impurity vs effective alpha for training set")

In [ ]:
#Tamara
model = tree.DecisionTreeClassifier(ccp_alpha=0.002)
model.fit(X_train[selected_columns], y_train)

test_predictions = model.predict(X_test[selected_columns]).round().astype(int)
print(accuracy_score(y_test, test_predictions))
mean_absolute_error(y_test, test_predictions)

In [ ]:
# Tamara
fig, ax = plt.subplots(figsize = (20,10))
tree.plot_tree(model, ax=ax,
               feature_names=selected_columns,
               class_names= ["Rain", "no Rain"],
               filled = True,
               rounded = True,
               precision = 2,
               fontsize = 10);

In [ ]:
# Tamara
clfs = []
for ccp_alpha in ccp_alphas_part:
    clf = tree.DecisionTreeClassifier(ccp_alpha=ccp_alpha)
    clf.fit(X_train, y_train)
    clfs.append(clf)
print("Number of nodes in the last tree is: {} with ccp_alpha: {}".format(
      clfs[-1].tree_.node_count, ccp_alphas[-1]))

In [ ]:
# Tamara
train_scores = [clf.score(X_train, y_train) for clf in clfs]
test_scores = [clf.score(X_test, y_test) for clf in clfs]

fig, ax = plt.subplots()
ax.set_xlabel("alpha")
ax.set_ylabel("accuracy")
ax.set_title("Accuracy vs alpha for training and testing sets")
ax.plot(ccp_alphas_part, train_scores, marker='o', label="train",
        drawstyle="steps-post")
ax.plot(ccp_alphas_part, test_scores, marker='o', label="test",
        drawstyle="steps-post")
ax.legend()
plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)

plt.show()

# Booklet 2 Neuronale Netze Aufgabe 2

### Using RandomSearch to find the perfect Hyperparameter combination

In [ ]:
#wrap function around model creation so it can be used for grid search
def build_model(n_hidden = 1, n_neurons=30, learning_rate=0.1, activation_function='relu', dropout_prop=0.25):
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(2, activation='relu'))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation=activation_function))
    model.add(keras.layers.Dense(1))
    model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    model.add(keras.layers.Dropout(dropout_prop))
    return model

nn = keras.wrappers.scikit_learn.KerasClassifier(build_model)

In [ ]:
params = {
    'n_hidden': [0,1,2,3,4],
    'n_neurons': [1,3,5,10,20,50,100],
    'learning_rate': [0.1, 0.05, 0.01],
    'activation_function': ['relu', 'sigmoid', 'elu'],
    'dropout_prop': [0, 0.25, 0.5] 
}

In [ ]:
random_search = RandomizedSearchCV(nn, params, n_iter=20)
random_search.fit(X_train.values, 
                  y_train.values,
                  validation_data=(X_test.values, y_test.values),
                  callbacks=[keras.callbacks.EarlyStopping(patience=6)],
                  batch_size=32,
                  epochs=100
                 )

In [ ]:
random_search.best_params_

## Building the winner Model

In [ ]:
#dont need this anymore...
train_dataset = tf.data.Dataset.from_tensor_slices((X_train.values, y_train.values))
train_dataset = train_dataset.shuffle(len(X_train)).batch(32)

test_dataset = tf.data.Dataset.from_tensor_slices((X_test.values, y_test.values))
test_dataset = test_dataset.shuffle(len(X_test)).batch(len(X_test))

In [ ]:
nn = keras.models.Sequential()
nn.add(keras.layers.Dense(2, activation='elu'))
for layer in range(4):
        nn.add(keras.layers.Dense(3, activation='elu'))
nn.add(keras.layers.Dense(1))
    
nn.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.1),
           loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
           metrics=['accuracy'])

In [ ]:
nn.fit(train_dataset,
       validation_steps=1,
       validation_data=test_dataset,
       batch_size=32,
       callbacks=[keras.callbacks.EarlyStopping(patience=6)],
       epochs=50)

# Booklet 3 Ensemblemethoden

In [ ]:
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
import matplotlib

## Univariat Grid Search

In [ ]:
params_bagging = [{'n_estimators': [10, 50, 100]},
                   {"base_estimator__criterion": ["gini", "entropy"]},
                   {"base_estimator__max_depth": [None, 3,5,10, 15, 20, 25, 50]},
                   {"base_estimator__min_samples_split": [2,5,10,20,30,40]},#The minimum number of samples required to split an internal node:
                   {"base_estimator__min_samples_leaf": [1,2,5,10,20,40]},#The minimum number of samples required to be at a leaf node
                   {"base_estimator__min_weight_fraction_leaf": [0, 0.2, 0.4, 0.5]},#The minimum weighted fraction of the sum total of weights (of all the input samples) required to be at a leaf node. 
                   {"base_estimator__max_features": [5,10,15,20, 25, None]},
                   {"base_estimator__max_leaf_nodes": [None, 2 ,10,100,150,200,300,500]},
                   {"base_estimator__min_impurity_decrease": [0.0, 0.1, 0.2, 0.5, 0.8, 1, 2, 3]},
                   {"base_estimator__min_impurity_split": [0.0, 0.1, 0.2, 0.5, 0.8, 1, 2, 3]} #dericated option
                  ]

params_adaboost = [{'n_estimators': [10, 50, 100]},
                   {'learning_rate': [0.1, 0.5, 1]},
                   {"base_estimator__criterion": ["gini", "entropy"]},
                   #{"base_estimator__max_depth": [None, 3,5,10, 15, 20, 25, 50]},
                   {"base_estimator__min_samples_split": [2,5,10,20,30,40]},#The minimum number of samples required to split an internal node:
                   {"base_estimator__min_samples_leaf": [1,2,5,10,20,40]},#The minimum number of samples required to be at a leaf node
                   {"base_estimator__min_weight_fraction_leaf": [0, 0.2, 0.4, 0.5]},#The minimum weighted fraction of the sum total of weights (of all the input samples) required to be at a leaf node. 
                   {"base_estimator__max_features": [1,2,10,15,20, None]},
                   {"base_estimator__max_leaf_nodes": [None,2,5,10,100,150,200]},
                   {"base_estimator__min_impurity_decrease": [0.0, 0.1, 0.2, 0.5, 0.8, 1, 2, 3]},
                   {"base_estimator__min_impurity_split": [0.0, 0.1, 0.2, 0.5, 0.8, 1, 2, 3]} #dericated option
                  ]

params_random_forest = [{'n_estimators': [10, 50, 100, 200, 500]},
                   {"criterion": ["gini", "entropy"]},
                   {"max_depth": [None, 3,5,10, 15, 20, 25, 50]},
                   #{"min_samples_split": [2,5,10,20,30,40]},#The minimum number of samples required to split an internal node:
                   {"min_samples_leaf": [1,2,5,10,20,40]},#The minimum number of samples required to be at a leaf node
                   {"min_weight_fraction_leaf": [0, 0.2, 0.4, 0.5]},#The minimum weighted fraction of the sum total of weights (of all the input samples) required to be at a leaf node. 
                   {"max_features": [5,10,15,20, 25, None]},
                   {"max_leaf_nodes": [None, 2 ,10,100,150,200,300,500]},
                   {"min_impurity_decrease": [0.0, 0.1, 0.2, 0.5, 0.8, 1, 2, 3]},
                   {"min_impurity_split": [0.0, 0.1, 0.2, 0.5, 0.8, 1, 2, 3]} #dericated option
                  ]

params_tree = [{"criterion": ["gini", "entropy"]},
                   #{"max_depth": [None, 3,5,10, 15, 20, 25, 50]},
                   {"min_samples_split": [2,5,10,20,30,40]},#The minimum number of samples required to split an internal node:
                   {"min_samples_leaf": [1,2,5,10,20,40]},#The minimum number of samples required to be at a leaf node
                   {"min_weight_fraction_leaf": [0, 0.2, 0.4, 0.5]},#The minimum weighted fraction of the sum total of weights (of all the input samples) required to be at a leaf node. 
                   {"max_features": [5,10,15,20, 25, None]},
                   {"max_leaf_nodes": [None, 2 ,10,100,150,200,300,500]},
                   {"min_impurity_decrease": [0.0, 0.1, 0.2, 0.5, 0.8, 1, 2, 3]},
                   {"min_impurity_split": [0.0, 0.1, 0.2, 0.5, 0.8, 1, 2, 3]} #dericated option
                  ]



In [ ]:
#looping through grid ourselfes for better interpretation of outputs.
for param in params_random_forest:
    
    bagging_classifier = BaggingClassifier( 
        base_estimator=DecisionTreeClassifier(),
        bootstrap=True, #replace training samples
        n_jobs=-1, #use all available cores
        random_state=42,
  #      n_estimators=100
    )
    
    bagging_gs = GridSearchCV(bagging_classifier, param, cv=10)
    bagging_gs.fit(X_train, y_train)
    print(param)
    print(bagging_gs.best_params_)
    print(bagging_gs.best_score_)
    print(bagging_gs.cv_results_['mean_test_score'])
    print(" ")

In [ ]:
start_time = time.time()

for param in params_random_forest:
    
    adaboost_classifier = AdaBoostClassifier(
        base_estimator=DecisionTreeClassifier(
        max_depth=5, 
        max_features=25,
            
            min_impurity_decrease=0.0005, 
            min_samples_split= 10
        ),
        random_state=42,
        
    )
    
    adaboost_gs = GridSearchCV(adaboost_classifier, param, cv=10)
    adaboost_gs.fit(X_train, y_train)
    print(param)
    print(adaboost_gs.best_params_)
    print(adaboost_gs.best_score_)
    print(adaboost_gs.cv_results_['mean_test_score'])
    print(" ")
    
elapsed_time = time.time() - start_time
print(elapsed_time)

In [ ]:
# multivariat parameter space
params_tree = [{"max_depth": [None,3,5,7],
     "min_samples_leaf": [50, 100, 200]
              }]

params_random_forest = [{
    "base_estimator__max_depth": [None,3, 5, 10],
    "base_estimator__min_samples_split": [5, 10],
    "base_estimator__max_features": [None, 5, 10, 25],
    "base_estimator__min_impurity_decrease": [0.0001, 0.0005]
              }]

In [ ]:
start_time = time.time()

for param in params_random_forest:
    
    tree_classifier = DecisionTreeClassifier( 
        random_state=42,
    )

    
    tree_gs = GridSearchCV(tree_classifier, params_random_forest, cv=10)
    tree_gs.fit(X_train, y_train)
    print(param)
    print(tree_gs.best_params_)
    print(tree_gs.best_score_)
    print(tree_gs.cv_results_['mean_test_score'])
    print(" ")
    
elapsed_time = time.time() - start_time
print(elapsed_time)

In [ ]:
start_time = time.time()

for param in params_random_forest:
    
    random_forest_classifier = RandomForestClassifier(  
        bootstrap=True, #replace training samples
        random_state=42,
        n_jobs=-1, #use all available cores,
        n_estimators=100,
    )

    
    random_forest_gs = GridSearchCV(random_forest_classifier, param, cv=10)
    random_forest_gs.fit(X_train, y_train)
    print(param)
    print(random_forest_gs.best_params_)
    print(random_forest_gs.best_score_)
    print(random_forest_gs.cv_results_['mean_test_score'])
    print(" ")
    
elapsed_time = time.time() - start_time
print(elapsed_time)

### Optimized Bagging Classifier

In [ ]:
# Ist das hier nicht im Prinzip das Gleiche wie ein Random Forest mit splitter="random"?
start_time = time.time()

bagging_classifier = BaggingClassifier( 
    base_estimator=DecisionTreeClassifier(
     max_features=5, 
        min_impurity_decrease=0.0001, 
        min_samples_split=10
    ),
    bootstrap=True, #replace training samples
    n_jobs=-1, #use all available cores
    random_state=42,
    n_estimators=100
)
bagging_classifier.fit(X_train, y_train)

test_predictions = bagging_classifier.predict(X_test).round().astype(int)
print(accuracy_score(y_test, test_predictions))
mean_absolute_error(y_test, test_predictions)

elapsed_time = time.time() - start_time
print(elapsed_time)

### Optimized AdaBoost Classifier

In [ ]:
start_time = time.time()
adaboost_classifier = AdaBoostClassifier(
    base_estimator=DecisionTreeClassifier(
        max_depth=20, 
        #max_features=25,
            
         #   min_impurity_decrease=0.0005, 
          #  min_samples_split= 10
        ),
    n_estimators=100,
    random_state=42
)

adaboost_classifier.fit(X_train, y_train)

test_predictions = adaboost_classifier.predict(X_test).round().astype(int)
print(accuracy_score(y_test, test_predictions))
mean_absolute_error(y_test, test_predictions)

elapsed_time = time.time() - start_time
print(elapsed_time)

### Optimized Random Forest

In [ ]:
random_forest_classifier = RandomForestClassifier(  
    bootstrap=True, #replace training samples
    random_state=42,
    n_jobs=-1, #use all available cores,
    min_impurity_decrease=0.00005,
    min_samples_split=5,
    n_estimators=100
)

random_forest_classifier.fit(X_train, y_train)

test_predictions = random_forest_classifier.predict(X_test).round().astype(int)
print(accuracy_score(y_test, test_predictions))
mean_absolute_error(y_test, test_predictions)

# Feature importances with random forest
for name, score in zip(X_train.columns, random_forest_classifier.feature_importances_):
    print(name, score)

### Optimized Tree

In [ ]:
tree_classifier = DecisionTreeClassifier( 
    random_state=42,
    min_impurity_decrease=0.0002,
    min_samples_split=6
    )

tree_classifier.fit(X_train, y_train)

test_predictions = tree_classifier.predict(X_test).round().astype(int)
print(accuracy_score(y_test, test_predictions))
mean_absolute_error(y_test, test_predictions)

### Visualisation of GridSearch results

In [ ]:
parameter = {"max_depth": [3,5,10, 15, 20, 25, 50,100]}
#parameter = {"min_impurity_decrease": [0.0001, 0.001, 0.003, 0.005, 0.01, 0.05, 0.1, 0.2]}
parameter_ = {"base_estimator__max_depth": [3,5,10, 15, 20, 25, 50,100]}

## Tree
tree_classifier = DecisionTreeClassifier( 
    random_state=42,
)

tree_gs = GridSearchCV(tree_classifier, parameter, cv=10, return_train_score=True)
tree_gs.fit(X_train, y_train)

results_tree = tree_gs.cv_results_
test_scores_tree = results_tree['mean_test_score']
train_scores_tree = results_tree['mean_train_score']


random_forest_classifier = RandomForestClassifier(  
    bootstrap=True, #replace training samples
    random_state=42,
    n_jobs=-1, #use all available cores,
)

## Forest
random_forest_gs = GridSearchCV(random_forest_classifier, parameter, cv=10, return_train_score=True)
random_forest_gs.fit(X_train, y_train)

results_forest = random_forest_gs.cv_results_
test_scores_forest = results_forest['mean_test_score']
train_scores_forest = results_forest['mean_train_score']

## bagging
bagging_classifier = BaggingClassifier( 
    base_estimator=DecisionTreeClassifier(),
    bootstrap=True, #replace training samples
    n_jobs=-1, #use all available cores
    random_state=42
)

bagging_gs = GridSearchCV(bagging_classifier, parameter_, cv=10, return_train_score=True)
bagging_gs.fit(X_train, y_train)

results_bagging = bagging_gs.cv_results_
test_scores_bagging = results_bagging['mean_test_score']
train_scores_bagging = results_bagging['mean_train_score']

## Adaboost
adaboost_classifier = AdaBoostClassifier(
    base_estimator=DecisionTreeClassifier(),
    random_state=42
)

adaboost_gs = GridSearchCV(adaboost_classifier, parameter_, cv=10, return_train_score=True)
adaboost_gs.fit(X_train, y_train)

results_adaboost = adaboost_gs.cv_results_
test_scores_adaboost = results_adaboost['mean_test_score']
train_scores_adaboost = results_adaboost['mean_train_score']


X_axis = np.array(results_tree['param_max_depth'].data, dtype=float)

In [ ]:
results_tree

In [ ]:
best_index_tree = np.nonzero(results_tree['rank_test_score'] == 1)[0][0]
best_score_tree = results_tree['mean_test_score'][best_index_tree]

best_index_adaboost = np.nonzero(results_adaboost['rank_test_score'] == 1)[0][0]
best_score_adaboost = results_adaboost['mean_test_score'][best_index_adaboost]

best_index_forest = np.nonzero(results_forest['rank_test_score'] == 1)[0][0]
best_score_forest = results_forest['mean_test_score'][best_index_forest]

best_index_bagging = np.nonzero(results_bagging['rank_test_score'] == 1)[0][0]
best_score_bagging = results_bagging['mean_test_score'][best_index_bagging]

In [ ]:
matplotlib.rcParams['figure.figsize'] = (10.0, 8.0)
fig1, ax1 = plt.subplots()

ax1.plot(X_axis, train_scores_tree, color='blue', alpha=0.7, linestyle='dashed' )
ax1.plot(X_axis, test_scores_tree, color='blue')

ax1.plot(X_axis, train_scores_forest, color = 'green', alpha=0.7, linestyle='dashed')
ax1.plot(X_axis, test_scores_forest, color = 'green')

ax1.plot(X_axis, train_scores_adaboost, color = 'red', alpha=0.7, linestyle='dashed')
ax1.plot(X_axis, test_scores_adaboost, color = 'red')

ax1.plot(X_axis, train_scores_bagging, color = 'purple', alpha=0.7, linestyle='dashed')
ax1.plot(X_axis, test_scores_bagging, color = 'purple')

ax1.set_xscale('log')
ax1.set_xticks([3,5,10,20,50,100])
ax1.get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
ax1.set_xlabel("max_depth", fontsize=14)
ax1.set_ylabel('Accuracy', fontsize=14)
plt.legend(['train tree','test tree', 'train forest', 'test forest','train adaboost', 'test adaboost', 'train bagging', 'test bagging'], fontsize=11)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

ax1.plot([X_axis[best_index_tree], ] * 2, [0, best_score_tree], linewidth=1,
        linestyle='-.', color='blue', marker='x', markeredgewidth=3, ms=8)
ax1.annotate("%0.3f" % best_score_tree,
            (X_axis[best_index_tree], best_score_tree + 0.005), backgroundcolor="w", fontsize=12)

ax1.plot([X_axis[best_index_adaboost], ] * 2, [0, best_score_adaboost], linewidth=1,
        linestyle='-.', color='red', marker='x', markeredgewidth=3, ms=8)
ax1.annotate("%0.3f" % best_score_adaboost,
            (X_axis[best_index_adaboost], best_score_adaboost + 0.005), backgroundcolor="w", fontsize=12)

ax1.plot([X_axis[best_index_forest], ] * 2, [0, best_score_forest], linewidth=1,
        linestyle='-.', color='green', marker='x', markeredgewidth=3, ms=8)
ax1.annotate("%0.3f" % best_score_forest,
            (X_axis[best_index_forest], best_score_forest + 0.005), backgroundcolor="w", fontsize=12)

ax1.plot([X_axis[best_index_bagging], ] * 2, [0, best_score_bagging], linewidth=1,
        linestyle='-.', color='purple', marker='x', markeredgewidth=3, ms=8)
ax1.annotate("%0.3f" % best_score_bagging, 
            (X_axis[best_index_bagging], best_score_bagging + 0.005), backgroundcolor="w", fontsize=12)

ax1.set_ylim(0.75,1)

plt.savefig('grid_search_max_depth.png')

# SVM Booklet Teil 4
**Note**: Hab keine Ahnung was hier genau passiert. Aber es funktioniert vom Code her.
### Parameters of SVM
- C: Sets the margin. Kleines C führt zu einer große margin (inversely proportional). Große Margin regularisiert mehr, also tritt eher underfitting auf. Großes C overfittetet eher.
- C kontrolliert den Bias-Varianze Tradeoff, indem es die tolerierte Missklassifikation und damit die Komplexität der Trennung bestimmt. 

In [ ]:
from sklearn.svm import SVC

### linear Kernel
$\langle x,x^{'}\rangle$

In [ ]:
svm_linear = SVC(kernel="linear", C=1)
svm_linear.fit(X_train, y_train)

test_predictions = svm_linear.predict(X_test).round().astype(int)
print(accuracy_score(y_test, test_predictions))
mean_absolute_error(y_test, test_predictions)

### Polynominal Kernel
$(\gamma\langle x,x^{'}\rangle + r)^{d}$

$d$ : degree

$r$: coef0

In [ ]:
svm_poly = SVC(kernel="poly", degree=5,C=1)
svm_poly.fit(X_train, y_train)

test_predictions = svm_poly.predict(X_test).round().astype(int)
print(accuracy_score(y_test, test_predictions))
mean_absolute_error(y_test, test_predictions)

### Gaussian RBF Kernel
$exp(-\gamma ||x-x^{'}||^{2})$

$\gamma$: gamma (>0)

$\gamma$ definiert den Einfluss der einzelenen Trainingsdaten
Je größer $\gamma$ ist, desto näher müssen andere Trainingsdatenpunkte sein, um einen Effekt zu haben
$\rightarrow \gamma$ gibt invertiert den Einfluss-Radius der Datenpunkte an, die als Support Vectors bestimmt wurden (vom Modell). 

In [ ]:
svm_rbf = SVC(kernel="rbf", gamma=5, C=0.01)
svm_rbf.fit(X_train, y_train)

test_predictions = svm_rbf.predict(X_test).round().astype(int)
print(accuracy_score(y_test, test_predictions))
mean_absolute_error(y_test, test_predictions)

### Sigmoid Kernel
$tanh(\gamma \langle x,x^{'}\rangle) + r)$

$r$: coef0

In [ ]:
svm_sigmoid = SVC(kernel="sigmoid", C=0.01)
svm_sigmoid.fit(X_train, y_train)

test_predictions = svm_sigmoid.predict(X_test).round().astype(int)
print(accuracy_score(y_test, test_predictions))
mean_absolute_error(y_test, test_predictions)

### precomputed Kernel

In [ ]:
gram_train = np.dot(X_train, X_train.T)
gram_test = np.dot(X_test, X_train.T)

svm_precomputed = SVC(kernel="precomputed", C=0.01)

svm_precomputed.fit(gram_train, y_train)

test_predictions = svm_precomputed.predict(gram_test).round().astype(int)
print(accuracy_score(y_test, test_predictions))
mean_absolute_error(y_test, test_predictions)